Boosting without oversampling

In [66]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import catboost
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


In [38]:
column_names = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 
                'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 
                'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 
                'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 
                'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 
                'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 
                'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'class', 'data']
KDD_train = pd.read_csv('KDDTrain.csv', header=None, names=column_names)
KDD_test = pd.read_csv('KDDTest.csv', header=None, names=column_names)
print(KDD_test.head())
KDD_train.head()

   duration protocol_type   service  flag  src_bytes  dst_bytes  land  \
0         0           tcp   private   REJ          0          0     0   
1         0           tcp   private   REJ          0          0     0   
2         2           tcp  ftp_data    SF      12983          0     0   
3         0          icmp     eco_i    SF         20          0     0   
4         1           tcp    telnet  RSTO          0         15     0   

   wrong_fragment  urgent  hot  ...  dst_host_same_srv_rate  \
0               0       0    0  ...                    0.04   
1               0       0    0  ...                    0.00   
2               0       0    0  ...                    0.61   
3               0       0    0  ...                    1.00   
4               0       0    0  ...                    0.31   

   dst_host_diff_srv_rate  dst_host_same_src_port_rate  \
0                    0.06                         0.00   
1                    0.06                         0.00   
2       

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class,data
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [39]:
encoder = LabelEncoder()
for column in KDD_train.columns:
    KDD_train[column] = encoder.fit_transform(KDD_train[column])

for column in KDD_test.columns:
    KDD_test[column] = encoder.fit_transform(KDD_test[column])

KDD_train.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class,data
0,0,1,20,9,482,0,0,0,0,0,...,17,3,17,0,0,0,5,0,11,20
1,0,2,44,9,143,0,0,0,0,0,...,0,60,88,0,0,0,0,0,11,15
2,0,1,49,5,0,0,0,0,0,0,...,10,5,0,0,100,99,0,0,9,19
3,0,1,24,9,229,5608,0,0,0,0,...,100,0,3,4,3,1,0,1,11,21
4,0,1,24,9,196,405,0,0,0,0,...,100,0,0,0,0,0,0,0,11,21


In [40]:
#train data
KDD_ip = KDD_train.drop('class', axis='columns')
KDD_op = KDD_train[['class']]
#test data
KDD_testip = KDD_test.drop('class', axis='columns')
KDD_testop = KDD_test[['class']]

Cat Boost

In [41]:
cat_features = []
cat_model = catboost.CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, loss_function='MultiClass')
cat_model.fit (KDD_ip, KDD_op, cat_features=cat_features)

0:	learn: 1.3459043	total: 415ms	remaining: 41s
1:	learn: 1.1082211	total: 829ms	remaining: 40.6s
2:	learn: 0.9420264	total: 1.24s	remaining: 40s
3:	learn: 0.8167433	total: 1.67s	remaining: 40.1s
4:	learn: 0.7121953	total: 2.07s	remaining: 39.4s
5:	learn: 0.6266368	total: 2.48s	remaining: 38.9s
6:	learn: 0.5552734	total: 2.91s	remaining: 38.7s
7:	learn: 0.4955593	total: 3.31s	remaining: 38.1s
8:	learn: 0.4448053	total: 3.73s	remaining: 37.7s
9:	learn: 0.3994238	total: 4.14s	remaining: 37.3s
10:	learn: 0.3597790	total: 4.54s	remaining: 36.7s
11:	learn: 0.3245403	total: 4.94s	remaining: 36.3s
12:	learn: 0.2948236	total: 5.35s	remaining: 35.8s
13:	learn: 0.2672014	total: 5.75s	remaining: 35.3s
14:	learn: 0.2441860	total: 6.17s	remaining: 34.9s
15:	learn: 0.2231582	total: 6.57s	remaining: 34.5s
16:	learn: 0.2040164	total: 6.98s	remaining: 34.1s
17:	learn: 0.1875954	total: 7.37s	remaining: 33.6s
18:	learn: 0.1723148	total: 7.77s	remaining: 33.1s
19:	learn: 0.1577375	total: 8.17s	remaining: 

In [42]:
cat_pred = cat_model.predict(KDD_testip)

In [67]:
report = classification_report(cat_pred, KDD_testop)
confusion = confusion_matrix(cat_pred, KDD_testop)
accuracy = accuracy_score(cat_pred, KDD_testop)
print("Accuracy:\n", accuracy)
print("Classification Report:\n", report)
print("Confusion Matrix:\n", confusion)

Accuracy:
 0.02479595457771469
Classification Report:
               precision    recall  f1-score   support

           0       0.02      0.04      0.03       448
           1       0.00      0.00      0.00         9
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00       168
           6       0.00      0.00      0.00         4
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00      5275
          10       0.00      0.00      0.00       364
          11       0.54      0.04      0.08     13267
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00        49
          15       0.00      0.00      0.00       258
          16       0.00   

C:\Users\arvin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\arvin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\arvin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetri

Light Boost

In [44]:
light_model = LGBMClassifier(learning_rate=0.1, n_estimators=100, max_depth=6, objective='multiclass')


In [45]:
light_model.fit (KDD_ip, KDD_op)

C:\Users\arvin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\arvin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012097 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3248
[LightGBM] [Info] Number of data points in the train set: 125973, number of used features: 39
[LightGBM] [Info] Start training from score -4.881065
[LightGBM] [Info] Start training from score -8.342625
[LightGBM] [Info] Start training from score -9.664381
[LightGBM] [Info] Start training from score -7.773531
[LightGBM] [Info] Start training from score -9.345928
[LightGBM] [Info] Start training from score -3.555412
[LightGBM] [Info] Start training from score -8.853451
[Ligh

LGBMClassifier(max_depth=6, objective='multiclass')

In [46]:
light_pred = light_model.predict(KDD_testip)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [68]:
report = classification_report(light_pred, KDD_testop)
confusion = confusion_matrix(light_pred, KDD_testop)
accuracy = accuracy_score(light_pred, KDD_testop)
print("Accuracy:\n", accuracy)
print("Classification Report:\n", report)
print("Confusion Matrix:\n", confusion)

Accuracy:
 0.03211497515968772
Classification Report:
               precision    recall  f1-score   support

           0       0.23      0.16      0.19      1060
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00       642
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00      2949
          10       0.00      0.00      0.00       133
          11       0.56      0.04      0.07     14897
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00       218
          16       0.00   

C:\Users\arvin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\arvin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\arvin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetri

Now performing Oversampling

In [48]:
KDD_op.value_counts()

class
11       67343
9        41214
17        3633
5         3599
15        2931
18        2646
10        1493
0          956
20         892
21         890
14         201
3           53
1           30
22          20
6           18
4           11
16          10
7            9
2            8
8            7
13           4
12           3
19           2
Name: count, dtype: int64

In [49]:
from imblearn.over_sampling import SMOTE

In [59]:
smote = SMOTE(sampling_strategy='minority', k_neighbors=1)

In [60]:
X_smt, y_smt = smote.fit_resample(KDD_ip, KDD_op)

In [ ]:
light_model.fit (X_smt, y_smt)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000541 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 95
[LightGBM] [Info] Number of data points in the train set: 37498, number of used features: 10
[LightGBM] [Info] Start training from score -1.099119
[LightGBM] [Info] Start training from score -1.263528
[LightGBM] [Info] Start training from score -3.436978
[LightGBM] [Info] Start training from score -4.291767
[LightGBM] [Info] Start training from score -5.999443
[LightGBM] [Info] Start training fro

C:\Users\arvin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\arvin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

LGBMClassifier(max_depth=6, objective='multiclass')

In [61]:
light_pred = light_model.predict(KDD_testip)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [69]:
report = classification_report(light_pred, KDD_testop)
confusion = confusion_matrix(light_pred, KDD_testop)
accuracy = accuracy_score(light_pred, KDD_testop)
print("Accuracy:\n", accuracy)
print("Classification Report:\n", report)
print("Confusion Matrix:\n", confusion)

Accuracy:
 0.03211497515968772
Classification Report:
               precision    recall  f1-score   support

           0       0.23      0.16      0.19      1060
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00       642
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00      2949
          10       0.00      0.00      0.00       133
          11       0.56      0.04      0.07     14897
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00       218
          16       0.00   

C:\Users\arvin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\arvin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\arvin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetri

In [63]:
cat_model.fit (X_smt, y_smt, cat_features=cat_features)

0:	learn: 1.2974350	total: 635ms	remaining: 1m 2s
1:	learn: 1.0798563	total: 1.29s	remaining: 1m 3s
2:	learn: 0.9089423	total: 1.97s	remaining: 1m 3s
3:	learn: 0.7849734	total: 2.56s	remaining: 1m 1s
4:	learn: 0.6856138	total: 3.15s	remaining: 59.8s
5:	learn: 0.6032415	total: 3.73s	remaining: 58.4s
6:	learn: 0.5343679	total: 4.31s	remaining: 57.2s
7:	learn: 0.4769920	total: 4.92s	remaining: 56.6s
8:	learn: 0.4242727	total: 5.53s	remaining: 55.9s
9:	learn: 0.3808310	total: 6.15s	remaining: 55.4s
10:	learn: 0.3427836	total: 6.79s	remaining: 54.9s
11:	learn: 0.3092126	total: 7.46s	remaining: 54.7s
12:	learn: 0.2791833	total: 8.15s	remaining: 54.6s
13:	learn: 0.2532421	total: 8.73s	remaining: 53.6s
14:	learn: 0.2301264	total: 9.33s	remaining: 52.9s
15:	learn: 0.2091883	total: 9.93s	remaining: 52.1s
16:	learn: 0.1906290	total: 10.5s	remaining: 51.3s
17:	learn: 0.1739062	total: 11.1s	remaining: 50.5s
18:	learn: 0.1590530	total: 11.7s	remaining: 49.7s
19:	learn: 0.1458941	total: 12.2s	remaini

In [64]:
cat_pred = cat_model.predict(KDD_testip)

In [70]:
report = classification_report(cat_pred, KDD_testop)
confusion = confusion_matrix(cat_pred, KDD_testop)
accuracy = accuracy_score(cat_pred, KDD_testop)
print("Accuracy:\n", accuracy)
print("Classification Report:\n", report)
print("Confusion Matrix:\n", confusion)

Accuracy:
 0.02479595457771469
Classification Report:
               precision    recall  f1-score   support

           0       0.02      0.04      0.03       448
           1       0.00      0.00      0.00         9
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00       168
           6       0.00      0.00      0.00         4
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00      5275
          10       0.00      0.00      0.00       364
          11       0.54      0.04      0.08     13267
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00        49
          15       0.00      0.00      0.00       258
          16       0.00   

C:\Users\arvin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\arvin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\arvin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetri